<!-- <img src="media/titlepage.png" width="100%"> -->


# Tutorial on Trixi.jl at ICOSAHOM 2021

## Exercises: Linear advection

### Michael Schlottke-Lakemper, Hendrik Ranocha 


- Follow along at https://git.io/JcDM1
- Launch MyBinder at https://tinyurl.com/khs92bk2
  (https://mybinder.org/v2/gh/trixi-framework/tutorial-2021-icosahom/HEAD?filepath=exercises_linear_advection.ipynb)

## Further information on running this notebook

This introduction is available as a Jupyter notebook at https://github.com/trixi-framework/tutorial-2021-icosahom, including information how to set up everything. For more information about Trixi and how to use it, please visit [Trixi on GitHub](https://github.com/trixi-framework/Trixi.jl) or refer to the [official documentation](https://trixi-framework.github.io/Trixi.jl/stable/). 

This notebook was set up and tested with Julia v1.6.1 but may also work with other (newer) versions.

*Note:* If you change a variable in a later cell and then re-execute an earlier cell, the results might change unexpectedly. Thus if in doubt, re-run the entire notebook *in order*. The reason is that all cells in a Jupyter notebooks share a common variable space.

*Note:* This notebook is tested using Chromium. Most parts should also work for other browsers such as Firefox, but the videos used in the last demonstrations might not be displayed correctly.


## Authors and license

This material is distributed by Michael Schlottke-Lakemper and Hendrik Ranocha under the MIT license.

# Exercise 1: Modify an initial condition

The code below solves the linear advection equation

$$
    \partial_t u + \partial_{x_1} u + \partial_{x_2} u = 0
$$

with periodic boundary conditions in the domain $[-1, 1]^2$. The initial condition

$$
    u(0, x) = 1 + \frac{1}{2} \sin(2 \pi (x_1 + x_2))
$$

is defined in Trixi as `initial_condition_convergence_test`.

### Task

Find out how to implement your own initial condition, e.g.

$$
    u(0, x) = \sin(\pi x_1) \cos(\pi x_2)
$$

In [ ]:
using Trixi, OrdinaryDiffEq
using Plots: plot
using Printf

advectionvelocity = (1.0, 1.0)
equations = LinearScalarAdvectionEquation2D(advectionvelocity)

solver = DGSEM(polydeg=3, surface_flux=flux_lax_friedrichs)

# Create a uniformely refined mesh with periodic boundaries
coordinates_min = (-1.0, -1.0) # lower left corner of the square
coordinates_max = ( 1.0,  1.0) # upper right corner of the square
mesh_static = TreeMesh(coordinates_min, coordinates_max,
                       initial_refinement_level=4, n_cells_max=10^5)

# Create semidiscretization with all spatial discretization-related components
semi = SemidiscretizationHyperbolic(mesh_static, equations,
                                    initial_condition_convergence_test,
                                    solver)

# Create ODE problem from semidiscretization with time span from 0.0 to 1.0
ode = semidiscretize(semi, (0.0, 1.0))

# Evolve ODE problem in time using `solve` from OrdinaryDiffEq
sol = solve(ode, BS3(), saveat=range(ode.tspan..., length=6))

# Plot the solution at different times
plot(map(i -> plot(sol.u[i], semi, title=@sprintf("\$ t = %.2f \$", sol.t[i])), 
        eachindex(sol.t))..., size=(1200, 550))

In [ ]:
# Your code can be written here

<details>

<summary>

View a possible solution (click on this line to expand it if you really want)

</summary>
    
```julia
function my_initial_condition(x, t, equation::LinearScalarAdvectionEquation2D)
  # Store translated coordinate for easy use of exact solution
  x_trans = x - equation.advectionvelocity * t

  scalar = sin(pi * x[1]) * cos(pi * x[2])
  return SVector(scalar)
end
```
    
</details>


# Exercise 2: Modify a numerical flux

We have used the local Lax-Friedrichs (Rusanov) flux, which coincides with the Godunov flux for this simple test case. This numerical flux is implemented as `flux_lax_friedrichs` in Trixi.

### Task

Find out how to implement your own numerical flux for this specific test case. You may use your knowledge of the `advectionvelocity`. Given left and right states `u_ll, u_rr`, a general numerical flux in Cartesian coordinate `direction` may use a parameter `factor` to blend a purely central flux and a purely upwind flux.

In one space dimension, such a numerical flux for the advection equation with constant unit velocity may look like

$$
    f^\mathrm{num}(u_l, u_r) = \alpha u_l + (1 - \alpha) u_r
$$

In [ ]:
using Trixi, OrdinaryDiffEq
using Plots: plot
using Printf

advectionvelocity = (1.0, 1.0)
equations = LinearScalarAdvectionEquation2D(advectionvelocity)

solver = DGSEM(polydeg=3, surface_flux=flux_lax_friedrichs)

# Create a uniformely refined mesh with periodic boundaries
coordinates_min = (-1.0, -1.0) # lower left corner of the square
coordinates_max = ( 1.0,  1.0) # upper right corner of the square
mesh_static = TreeMesh(coordinates_min, coordinates_max,
                       initial_refinement_level=4, n_cells_max=10^5)

# Create semidiscretization with all spatial discretization-related components
semi = SemidiscretizationHyperbolic(mesh_static, equations,
                                    initial_condition_convergence_test,
                                    solver)

# Create ODE problem from semidiscretization with time span from 0.0 to 1.0
ode = semidiscretize(semi, (0.0, 1.0))

# Evolve ODE problem in time using `solve` from OrdinaryDiffEq
sol = solve(ode, BS3(), saveat=range(ode.tspan..., length=6))

# Plot the solution at different times
plot(map(i -> plot(sol.u[i], semi, title=@sprintf("\$ t = %.2f \$", sol.t[i])), 
        eachindex(sol.t))..., size=(1200, 550))

In [ ]:
# Your code can be written here

<details>

<summary>

View a possible solution (click on this line to expand it if you really want)

</summary>
    
```julia
@inline function my_numerical_flux(u_ll, u_rr, direction::Integer, equation::LinearScalarAdvectionEquation2D)
  a = equation.advectionvelocity[orientation]

  # We know that the advection velocity is positive. Thus, we can
  # pick the left state to get a pure upwind flux.
  return a * u_ll
end
```
    
</details>


# Exercise 3: Choose another time integration method

We have used the third-order method `BS3()` Of Bogacki and Shampine from OrdinaryDiffEq.jl, which is part of the DifferentialEquations.jl ecosystem in Julia.

### Task

1. Experiment with different time integration methods. Find out where you can obtain information about methods implemented in OrdinaryDiffEq.jl. Which are developed specifically for hyperbolic problems?
2. How can you modify the tolerances (absolute and relative) of methods with error based step size control?
3. How can you use time step control via a CFL condition in Trixi?

In [ ]:
using Trixi, OrdinaryDiffEq
using Plots: plot
using Printf

advectionvelocity = (1.0, 1.0)
equations = LinearScalarAdvectionEquation2D(advectionvelocity)

solver = DGSEM(polydeg=3, surface_flux=flux_lax_friedrichs)

# Create a uniformely refined mesh with periodic boundaries
coordinates_min = (-1.0, -1.0) # lower left corner of the square
coordinates_max = ( 1.0,  1.0) # upper right corner of the square
mesh_static = TreeMesh(coordinates_min, coordinates_max,
                       initial_refinement_level=4, n_cells_max=10^5)

# Create semidiscretization with all spatial discretization-related components
semi = SemidiscretizationHyperbolic(mesh_static, equations,
                                    initial_condition_convergence_test,
                                    solver)

# Create ODE problem from semidiscretization with time span from 0.0 to 1.0
ode = semidiscretize(semi, (0.0, 1.0))

# Evolve ODE problem in time using `solve` from OrdinaryDiffEq
sol = solve(ode, BS3(), saveat=range(ode.tspan..., length=6))

# Plot the solution at different times
plot(map(i -> plot(sol.u[i], semi, title=@sprintf("\$ t = %.2f \$", sol.t[i])), 
        eachindex(sol.t))..., size=(1200, 550))

In [ ]:
# Your code can be written here

<details>

<summary>

View some hints (click on this line to expand it if you really want)

</summary>

- Documentation of DiffEq: https://diffeq.sciml.ai/dev/
- Time integration methods for hyperbolic problems
  - [SSP methods](https://diffeq.sciml.ai/dev/solvers/ode_solve/#Explicit-Strong-Stability-Preserving-Runge-Kutta-Methods-for-Hyperbolic-PDEs-(Conservation-Laws))
  - [Low-storage methods](https://diffeq.sciml.ai/dev/solvers/ode_solve/#Low-Storage-Methods)
- Tolerances `abstol, reltol` via keyword arguments, cf. https://diffeq.sciml.ai/dev/basics/common_solver_opts/#Stepsize-Control
- Look at the documentation of the [`StepsizeCallback`](https://trixi-framework.github.io/Trixi.jl/stable/callbacks/#CFL-based-time-step-control)
    
</details>
